<a href="https://colab.research.google.com/github/PedroHON410/ETL-com-python-e-IA/blob/main/ETL_com_python_e_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EXTRAINDO OS DADOS**

In [14]:
#Importando bibliotecas
import pandas as pd
from google.colab import userdata

In [ ]:
#Importando os dados e transformando em Data Frame
df = pd.read_csv('comentarios.csv')
df_original = pd.DataFrame(df)
print(df_original.head())

   id_avaliacao                               comentario_avaliacao  \
0             1  Produto excelente, superou minhas expectativas...   
1             2  A qualidade é péssima. Descosturou no primeiro...   
2             3  Bom custo-benefício, mas demorou uma semana a ...   
3             4    Funciona perfeitamente. Estou muito satisfeita.   

   nota_avaliacao  
0              10  
1               1  
2               7  
3              10  


# **TRANSFORMANDO OS DADOS**

In [16]:
#Importando API da IA e criando a chave
from openai import OpenAI
API_KEY_OPENIA = userdata.get('API_KEY_OPENIA')
client = OpenAI(api_key = API_KEY_OPENIA)

In [ ]:
import json
#Criando o prompt para análise das avaliações e retorno da resposta da IA
def transformando_dados(comentario_avaliacao, nota_avaliacao):
  system_prompt = (
        "Você é um analista de dados especialista em e-commerce. Sua tarefa é analisar "
        "avaliações de clientes. Para cada texto de avaliação, você deve: "
        "a) Classificar o sentimento como: 'Positivo', 'Negativo' ou 'Neutra'. "
        "b) Identificar o tópico principal da crítica entre 'Qualidade do Produto', 'Entrega/Logística', 'Preço/Valor'"
        "c) Identificar a nota da avaliação e ranquear de acordo, sendo 1 a pior nota e 10 a melhor nota."
        "Você deve retornar a resposta APENAS no formato JSON. Não adicione nenhum texto extra. "
        "O formato JSON deve ter as chaves 'sentimento' e 'topico_principal'."
  )

  try:
    response = client.chat.completions.create(
      model="gpt-4o-mini",
      response_format={"type": "json_object"},
      messages=[
                {
                    "role": "system", "content": system_prompt
                  },
                  {
                      "role": "user", "content": f"Analise esta avaliação: '{comentario_avaliacao, nota_avaliacao}'"
                  }
              ],
    )
  #Retornando arquivo JSON
    json_string = response.choices[0].message.content
    return json.loads(json_string)
  except Exception as e:
    print(f"Erro ao processar avaliação: {e}")
    return None


In [ ]:
#aplicando a os dados na função e retornando
df_original['resultado'] = df_original.apply(lambda x: transformando_dados(x['comentario_avaliacao'], x['nota_avaliacao']), axis=1)
print(df_original[['comentario_avaliacao','nota_avaliacao', 'resultado']].head())


                                comentario_avaliacao  nota_avaliacao  \
0  Produto excelente, superou minhas expectativas...              10   
1  A qualidade é péssima. Descosturou no primeiro...               1   
2  Bom custo-benefício, mas demorou uma semana a ...               7   
3    Funciona perfeitamente. Estou muito satisfeita.              10   

                                           resultado  
0  {'sentimento': 'Positivo', 'topico_principal':...  
1  {'sentimento': 'Negativo', 'topico_principal':...  
2  {'sentimento': 'Neutra', 'topico_principal': '...  
3  {'sentimento': 'Positivo', 'topico_principal':...  


# **CARREGANDO OS DADOS**

In [ ]:
#Transformando os arquivos em um CSV
comentarios_melhorados = 'comentarios_melhorados.csv'
df_original.to_csv(comentarios_melhorados, index=False)
